In [179]:
import pandas as pd
from pathlib import Path
import plotly.express as px
from plotly.subplots import make_subplots

In [180]:
csv_dir = Path("./runs")

csv_files = list(csv_dir.glob("*.csv"))

dataframes = []
for file in csv_files:
    name = str(file).split("/")[-1].split(".")[0]
    df = pd.read_csv(file)
    df['id'] = name
    dataframes.append(df)

df = pd.concat(dataframes, ignore_index=True)

df['reward_100ma'] = df.groupby('id')['reward'].transform(lambda x: x.rolling(window=100).mean())
df['cumls_steps'] = df.groupby('id')['steps'].transform(lambda x: x.cumsum())
df['cumls_time'] = df.groupby('id')['time_seconds'].transform(lambda x: x.cumsum())
df = df.dropna()

df.head()

,episode,reward,time_seconds,steps,id,reward_100ma,cumls_steps,cumls_time
99,100,117.8436,0.0219,553,ppo_config1,-98.989481,26844,0.4919
100,101,13.1221,0.0229,1000,ppo_config1,-97.802021,27844,0.5148
101,102,-48.5307,0.0241,1000,ppo_config1,-97.777379,28844,0.5389
102,103,-143.3486,0.0241,109,ppo_config1,-99.228642,28953,0.5630
103,104,-85.4607,0.0243,316,ppo_config1,-97.536878,29269,0.5873


In [187]:
# px.line(df, x='episode', y='reward_100ma', color='id', facet_col='id', facet_col_wrap=3, width=1600, height=800)

final_values = df.groupby('id')['reward_100ma'].last().sort_values(ascending=False)
category_orders = {'id': final_values.index.tolist()}

fig = px.line(df, x='episode', y='reward_100ma', color='id', category_orders=category_orders, width=1300, height=600)


fig.add_annotation(
    x=x_max, y=200, text="Human Level", showarrow=False,
    xanchor="left", xshift=5, yshift=10,  
    font=dict(color="black", size=12),
    row=1, col=1
)

fig.add_hline(
    y=200,
    line=dict(color="black", width=1.5, dash="dash"),
    row=1, col=1
)

In [176]:


final_values = df.groupby('id')['reward_100ma'].last().sort_values(ascending=False)
category_orders = {'id': final_values.index.tolist()}

dqn_reward = px.line(df, x='episode', y='reward_100ma', color='id', 
               title="Model Performance Comparison", width=1200, height=600,
               category_orders=category_orders)



In [178]:
fig = make_subplots(rows=2, cols=2, 
                   subplot_titles=("Reward (100ma)", "Cumulative Steps", "Cumulative Time"),
                   vertical_spacing=0.1,
                   column_widths=[0.5, 0.5],
                   row_heights=[0.65, 0.35],  
                   specs=[[{"colspan": 2}, None], [{}, {}]])

# Fix the typo in the filter: 'dnq' should be 'dqn'
fig_ppo_reward = px.line(df[df.id.str.contains('dqn')], x='episode', y='reward_100ma', color='id', title='DQN Reward (100ma)', width=1000, height=500)
fig_ppo_cumls_steps = px.line(df[df.id.str.contains('dqn')], x='episode', y='cumls_steps', color='id', title='DQN Cumulative Steps', width=1000, height=500)
fig_ppo_cumls_time = px.line(df[df.id.str.contains('dqn')], x='episode', y='cumls_time', color='id', title='DQN Cumulative Time', width=1000, height=500)

used_names = set()  
for trace in dqn_reward.data:
    if trace.name in used_names:
        trace.showlegend = False
    else:
        used_names.add(trace.name)
    fig.add_trace(trace, row=1, col=1)
    
for trace in fig_ppo_cumls_steps.data:
    trace.showlegend = False
    fig.add_trace(trace, row=2, col=1)
    
for trace in fig_ppo_cumls_time.data:
    trace.showlegend = False
    fig.add_trace(trace, row=2, col=2)

# Add reference lines with annotations
x_min = df[df.id.str.contains('dqn')]['episode'].min()
x_max = df[df.id.str.contains('dqn')]['episode'].max()

fig.add_annotation(
    x=x_max, y=200, text="Human Level", showarrow=False,
    xanchor="left", xshift=5, yshift=10,  
    font=dict(color="black", size=12),
    row=1, col=1
)

fig.add_hline(
    y=200,
    line=dict(color="black", width=1.5, dash="dash"),
    row=1, col=1
)

# Update axis titles for better clarity
fig.update_xaxes(title_text="Episode", row=2, col=1)
fig.update_xaxes(title_text="Episode", row=2, col=2)
fig.update_yaxes(title_text="Cumulative Steps", row=2, col=1)
fig.update_yaxes(title_text="Cumulative Time (s)", row=2, col=2)

fig.update_layout(
    height=850,
    width=1300,
    title_text="DQN Performance Report",
    hovermode="x unified", 
    margin=dict(t=80, l=50, r=50, b=50)
)